# PROBLEMA DELLA DIETA

## Obiettivo e Prerequisiti
L'obiettivo di questo problema di ottimizzazione è quello di definire una dieta, sulla base dei dati messi a disposizione riguardanti alimenti e relativi valori nutrizionali, al minor costo possibile.

In questa esercitazione si farà uso della libreria dedicata all'ottimizzazione matematica **PuLP**

## Descrizione del problema
Supponendo di essere il responsabile della definizione di una dieta per la mensa scolastica, è necessario assicurarsi che gli studenti mantengano un giusto equilibrio nutrizionale, tenendo conto di alcune restrizioni in termini di budget e varietà di cibo che deve essere presente nella dieta.

## Formulazione del modello
Da un punto di vista meramente numerico Il problema di ottimizzazione consiste nel minimizzare il costo del pranzo considerando dei vincoli (sul totale delle calorie o anche su ognuna delle componenti nutrizionali).
Quanto appena riportato si può formalizzare come segue:

\begin{equation}
c_{i} \text{: costo unitario del i-esimo alimento}
\end{equation}
\begin{equation}
f_{i} \text{: quantità del i-esimo alimento}
\end{equation}

\begin{equation}
\text{Min} \quad :\sum_{i=1..n}C_{i} \cdot f_{i}
\end{equation}

\begin{equation}
\text{subject to calorieLB <}\quad\sum_{i=1..n}Cal_{i} \cdot f_{i} \text{< calorieUB }
\end{equation}

\begin{equation}
\text{subject to proteineLB <}\quad\sum_{i=1..n}Pro_{i} \cdot f_{i} \text{< proteineUB }
\end{equation}

## Implementazione

### Lettura del dataset contenente i dati nutrizionali
Importing del file *dieta.xls*, il quale è stato reperito da Kaggle.

In [23]:
import pandas as pd
df = pd.read_excel("dieta.xls",nrows=64)

In [24]:
# Visualizzazione delle prime righe del dataset
df.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


## Creazione del PL

In [25]:
from pulp import *
# Creazione della variabile 'PuLP'. 
# Dal momento che si tratta di un problema di minimizzazione, occorre utilizzare 'LpMinimize'
prob = LpProblem("Diet Problem",LpMinimize)

C:\Users\polic\Anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Data Handling

In [26]:
# creazione di una lista contenente tutti gli alimenti
food_items = list(df['Foods'])

In [27]:
print("Gli alimenti da considerare in questo problema sono:\n"+"-"*100)
for f in food_items:
    print(f,end=', ')

Gli alimenti da considerare in questo problema sono:
----------------------------------------------------------------------------------------------------
Frozen Broccoli, Carrots,Raw, Celery, Raw, Frozen Corn, Lettuce,Iceberg,Raw, Peppers, Sweet, Raw, Potatoes, Baked, Tofu, Roasted Chicken, Spaghetti W/ Sauce, Tomato,Red,Ripe,Raw, Apple,Raw,W/Skin, Banana, Grapes, Kiwifruit,Raw,Fresh, Oranges, Bagels, Wheat Bread, White Bread, Oatmeal Cookies, Apple Pie, Chocolate Chip Cookies, Butter,Regular, Cheddar Cheese, 3.3% Fat,Whole Milk, 2% Lowfat Milk, Skim Milk, Poached Eggs, Scrambled Eggs, Bologna,Turkey, Frankfurter, Beef, Ham,Sliced,Extralean, Kielbasa,Prk, Cap'N Crunch, Cheerios, Corn Flks, Kellogg'S, Raisin Brn, Kellg'S, Rice Krispies, Special K, Oatmeal, Malt-O-Meal,Choc, Pizza W/Pepperoni, Taco, Hamburger W/Toppings, Hotdog, Plain, Couscous, White Rice, Macaroni,Ckd, Peanut Butter, Pork, Sardines in Oil, White Tuna in Water, Popcorn,Air-Popped, Potato Chips,Bbqflvr, Pretzels, Tortill

### Creazione dizionari
Costi per tutti gli alimenti

In [28]:
costs = dict(zip(food_items,df['Price/ Serving']))
costs

{'Frozen Broccoli': 0.16,
 'Carrots,Raw': 0.07,
 'Celery, Raw': 0.04,
 'Frozen Corn': 0.18,
 'Lettuce,Iceberg,Raw': 0.02,
 'Peppers, Sweet, Raw': 0.53,
 'Potatoes, Baked': 0.06,
 'Tofu': 0.31,
 'Roasted Chicken': 0.84,
 'Spaghetti W/ Sauce': 0.78,
 'Tomato,Red,Ripe,Raw': 0.27,
 'Apple,Raw,W/Skin': 0.24,
 'Banana': 0.15,
 'Grapes': 0.32,
 'Kiwifruit,Raw,Fresh': 0.49,
 'Oranges': 0.15,
 'Bagels': 0.16,
 'Wheat Bread': 0.05,
 'White Bread': 0.06,
 'Oatmeal Cookies': 0.09,
 'Apple Pie': 0.16,
 'Chocolate Chip Cookies': 0.03,
 'Butter,Regular': 0.05,
 'Cheddar Cheese': 0.25,
 '3.3% Fat,Whole Milk': 0.16,
 '2% Lowfat Milk': 0.23,
 'Skim Milk': 0.13,
 'Poached Eggs': 0.08,
 'Scrambled Eggs': 0.11,
 'Bologna,Turkey': 0.15,
 'Frankfurter, Beef': 0.27,
 'Ham,Sliced,Extralean': 0.33,
 'Kielbasa,Prk': 0.15,
 "Cap'N Crunch": 0.31,
 'Cheerios': 0.28,
 "Corn Flks, Kellogg'S": 0.28,
 "Raisin Brn, Kellg'S": 0.34,
 'Rice Krispies': 0.32,
 'Special K': 0.38,
 'Oatmeal': 0.82,
 'Malt-O-Meal,Choc': 0.52,
 

Dizionario per tutti gli alimenti relativamente a:
- Calorie
- Colesterolo
- Grassi
- Sodio
- Carboidrati
- Fibre
- Proteine
- Vitamina A
- Vitamina C
- Calcio
- Ferro

In [29]:
calories = dict(zip(food_items,df['Calories']))
cholesterol = dict(zip(food_items,df['Cholesterol mg']))
fat = dict(zip(food_items,df['Total_Fat g']))
sodium = dict(zip(food_items,df['Sodium mg']))
carbs = dict(zip(food_items,df['Carbohydrates g']))
fiber = dict(zip(food_items,df['Dietary_Fiber g']))
protein = dict(zip(food_items,df['Protein g']))
vit_A = dict(zip(food_items,df['Vit_A IU']))
vit_C = dict(zip(food_items,df['Vit_C IU']))
calcium = dict(zip(food_items,df['Calcium mg']))
iron = dict(zip(food_items,df['Iron mg']))

Dizionario di alimenti aventi **limite inferiore** = 0 e categoria Integer, ovvero la soluzione di ottimizzazione può assumere qualsiasi valore con numero intero maggiore di zero.

Si utilizza la categoria **Integer** perchè si può pensare ad una porzione di cibo solo come una quantità intera e non negativa.
Senza un'esplicita dichiarazione di questo limite, la soluzione potrebbe essere priva di senso in quanto il solver potrebbe provare a fornire quantità negative o reali per ridurre il costo totale.

In [30]:
# Creazione del dizionario 'food_vars' per contenere le variabili di riferimento
food_vars = LpVariable.dicts("Portion",food_items,lowBound=0,cat='Integer')


Dizionario per comprendere i cibi scelti o meno tramite i valori 0 o 1.

In [31]:
food_chosen = LpVariable.dicts("Chosen",food_items,0,1,cat='Integer')

## Definizione del PL

In [32]:
# Funzione obiettivo
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

## Vincoli
### Vincoli relativi ai valori nutrizionali al problema

In [33]:
# Calorie
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 1500.0, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 2500.0, "CalorieMaximum"

# Colesterolo
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) >= 30.0, "CholesterolMinimum"
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) <= 240.0, "CholesterolMaximum"

# Grassi
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 70.0, "FatMaximum"

# Sodio
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) >= 800.0, "SodiumMinimum"
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) <= 2000.0, "SodiumMaximum"

# Carboidrati
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 450.0, "CarbsMaximum"

# Fibre
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 125.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 250.0, "FiberMaximum"

# Proteine
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 60.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 100.0, "ProteinMaximum"

# Vitamina A
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) >= 1000.0, "VitaminAMinimum"
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) <= 10000.0, "VitaminAMaximum"

# Vitamina C
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) >= 400.0, "VitaminCMinimum"
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) <= 5000.0, "VitaminCMaximum"

# Calcio
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) >= 700.0, "CalciumMinimum"
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) <= 1500.0, "CalciumMaximum"

# Ferro
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) >= 10.0, "IronMinimum"
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) <= 40.0, "IronMaximum"

In [34]:
# loop per linkare food_vars e food_chosen
for f in food_items:
    prob += food_vars[f]>= food_chosen[f]*0.1
    prob += food_vars[f]<= food_chosen[f]*1e5

### Vincolo per sedano e broccoli
Presenza di al più 1 dei due alimenti in dieta

In [35]:
prob += food_chosen['Frozen Broccoli']+food_chosen['Celery, Raw']<=1

### Vincolo per alimenti proteici
Presenza di almeno 3 tipi di alimento proteico nella dieta

In [36]:
protein_choices = ['Beanbacn Soup,W/Watr','Bologna,Turkey','Frankfurter, Beef','Ham,Sliced,Extralean',
                  'Hamburger W/Toppings','Hotdog, Plain','Kielbasa,Prk','Neweng Clamchwd','Pizza W/Pepperoni',
                  'Poached Eggs','Pork','Roasted Chicken','Sardines in Oil','Scrambled Eggs','Vegetbeef Soup',
                   'White Tuna in Water']

In [37]:
prob += lpSum([food_chosen[p] for p in protein_choices]) >= 3.0

In [38]:
# Scrittura del problema costruito in un file '.lp'
prob.writeLP("DietProblem.lp")

[Chosen_2%_Lowfat_Milk,
 Chosen_3.3%_Fat,Whole_Milk,
 Chosen_Apple,Raw,W_Skin,
 Chosen_Apple_Pie,
 Chosen_Bagels,
 Chosen_Banana,
 Chosen_Beanbacn_Soup,W_Watr,
 Chosen_Bologna,Turkey,
 Chosen_Butter,Regular,
 Chosen_Cap'N_Crunch,
 Chosen_Carrots,Raw,
 Chosen_Celery,_Raw,
 Chosen_Cheddar_Cheese,
 Chosen_Cheerios,
 Chosen_Chicknoodl_Soup,
 Chosen_Chocolate_Chip_Cookies,
 Chosen_Corn_Flks,_Kellogg'S,
 Chosen_Couscous,
 Chosen_Crm_Mshrm_Soup,W_Mlk,
 Chosen_Frankfurter,_Beef,
 Chosen_Frozen_Broccoli,
 Chosen_Frozen_Corn,
 Chosen_Grapes,
 Chosen_Ham,Sliced,Extralean,
 Chosen_Hamburger_W_Toppings,
 Chosen_Hotdog,_Plain,
 Chosen_Kielbasa,Prk,
 Chosen_Kiwifruit,Raw,Fresh,
 Chosen_Lettuce,Iceberg,Raw,
 Chosen_Macaroni,Ckd,
 Chosen_Malt_O_Meal,Choc,
 Chosen_New_E_Clamchwd,W_Mlk,
 Chosen_Neweng_Clamchwd,
 Chosen_Oatmeal,
 Chosen_Oatmeal_Cookies,
 Chosen_Oranges,
 Chosen_Peanut_Butter,
 Chosen_Peppers,_Sweet,_Raw,
 Chosen_Pizza_W_Pepperoni,
 Chosen_Poached_Eggs,
 Chosen_Popcorn,Air_Popped,
 Chosen_

## Risoluzione del PL

In [39]:
# Non specificando il solver da utilizzare si lascia che la libreria PuLP decida il migliore sulla base
# della struttura del problema
prob.solve()

1

## Stampa dello stato della soluzione PL
Sebbene in questo caso lo stato sia ottimale, non è sempre detto che lo sia. 
Nel caso in cui il problema sia mal formulato o non ci siano informazioni sufficienti, la soluzione potrebbe essere infeasible o unbounded.

In [40]:
print("Status:", LpStatus[prob.status])

Status: Optimal


La **soluzione completa** contiene tutte le variabili comprese quelle con peso zero.
Di interesse sono solo quelle variabili che hanno coefficienti diversi da zero, cioè che dovrebbero essere incluse nel piano alimentare ottimale. Quindi, si procede a stampare solo se la quantità della variabile è positiva.

In [41]:
print("La dieta bilanciata ottimale (costo minimo) che considera i vincoli consiste in:\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0 and v.name[0]=='P':
        print(v.name, "=", v.varValue)

La dieta bilanciata ottimale (costo minimo) che considera i vincoli consiste in:
--------------------------------------------------------------------------------------------------------------
Portion_Bologna,Turkey = 1
Portion_Celery,_Raw = 32
Portion_Kielbasa,Prk = 1
Portion_Kiwifruit,Raw,Fresh = 8
Portion_Lettuce,Iceberg,Raw = 96
Portion_Popcorn,Air_Popped = 12
Portion_Sardines_in_Oil = 1
Portion_Tofu = 1


## Costo ottimale della dieta

In [42]:
print("Il costo totale della dieta bilanciata è: ${}".format(round(value(prob.objective),2)))

Il costo totale della dieta bilanciata è: $8.66
